In [1]:
import os 
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from keras import models
from keras import layers
from keras import optimizers
from keras.utils import normalize
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.
/Users/markkoerner/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


In [2]:
# from Prateeks NN notebook

# set the variables for the parent data folder and train file
dataFolder = '/Users/markkoerner/PythonProjects/BBDCActivityClassification/bbdc_2019_Bewegungsdaten/original_dataset/'
mainFolder = '/Users/markkoerner/PythonProjects/BBDCActivityClassification/'
trainData = 'train.csv'

# make a list of all the csv files in the parent and sub directories
result = list(Path(dataFolder).rglob("Subject*.[cC][sS][vV]"))

In [3]:
len(result)

8139

#### Feature Engineering

The following code does not work properly. The code after #windowing is supposed to create windows of 100 observations, return the mean of each window and then create a new array with all the means. As of now, a large portion of the new dataframe contains NaNs, so using the resulting feature array as input for a CNN will result in the model failing to converge. 

In [141]:
# Adaptation of Prateeks code for generating dataset

# Number of files to load
numFiles = 6385

# Maximum file size
maxlen = 8000

# NP Arrays to store the data and the label
labelArr = np.empty((numFiles, 1))
npArr = np.empty((numFiles, 160, 19))
indxArr = []

# read the training data into a dataframe
trainDf = pd.read_csv(dataFolder+trainData)
trainDf = trainDf[trainDf.Label != 'lay']
# factorize the labels
factor = pd.factorize(trainDf.Label)
trainDf.Label = factor[0]


cnt = 0
for eachFile in result:
  # Only read the dataset with labels available
    if sum(str(eachFile)[-35:] == trainDf.Datafile)>0:
        indxArr.append(str(eachFile)[-35:])
        file = np.loadtxt(str(eachFile), delimiter=',')
        #min max scale the inpuit
        file = (file - file.mean(axis=1, keepdims=True))
        
        #zero pad to conserve last values
        #padFile = np.zeros((50, 19))
        #file = np.concatenate((file, padFile))
        
        #windowing
        df = pd.DataFrame(file)
        emg = df.iloc[:, :4]
        other = df.iloc[::10, 4:]
        emg = emg ** 2
        emg = emg.rolling(100).mean().dropna(axis = 0, how = 'any').reset_index(drop = True)
        emg = emg.iloc[::50, :].reset_index(drop = True)
        emg = emg ** 2
        other = other.rolling(10).mean().dropna(axis = 0, how ='any').iloc[::5, :].reset_index(drop = True)
        df = pd.concat([emg, other], axis = 'columns')
        
        file = df.values
        #extend to max length
        file = (file - file.min(axis=1, keepdims=True)) / (file.max(axis = 1, keepdims = True) - file.min(axis = 1, keepdims = True))
        padFile = np.zeros((159, 19))
        file = np.concatenate((file, padFile))
        file = file[:160]
        
        data = np.expand_dims(file, axis=0)
        npArr[cnt] = data
        labelArr[cnt] = trainDf[trainDf.Datafile == str(eachFile)[-35:]].Label
        cnt += 1

In [57]:
#create test array

# Maximum file size
maxlen = 8000

# read the training data into a dataframe
testDf = pd.read_csv(dataFolder+'challenge.csv')

#number of files to read
numFiles = testDf.shape[0]

testArr = np.empty((numFiles, 160, 19))
indyArr = []

In [58]:
# same process for testing set
cnt = 0
for eachFile in result:
  # Only read the dataset with labels available
    if sum(str(eachFile)[-35:] == testDf.Datafile)>0:
        indyArr.append(str(eachFile)[-35:])
        file = np.loadtxt(str(eachFile), delimiter=',')
        #min max scale the inpuit
        file = (file - file.mean(axis=1, keepdims=True))
        
        #zero pad to conserve last values
        padFile = np.zeros((50, 19))
        file = np.concatenate((file, padFile))
        
        #windowing
        df = pd.DataFrame(file)
        emg = df.iloc[:, :4]
        other = df.iloc[::10, 4:]
        emg = emg ** 2
        emg = emg.rolling(100).mean().dropna(axis = 'index').iloc[::50, :].reset_index(drop = True)
        other = other.rolling(10).mean().dropna(axis = 'index').iloc[::5, :].reset_index(drop = True)
        df = pd.concat([emg, other], axis = 1)
        
        file = df.values
        #extend to max length
        file = (file - file.min(axis=1, keepdims=True)) / (file.max(axis = 1, keepdims = True) - file.min(axis = 1, keepdims = True))
        padFile = np.zeros((159, 19))
        file = np.concatenate((file, padFile))
        file = file[:160]
        
        data = np.expand_dims(file, axis=0)
        testArr[cnt] = data
        cnt += 1

In [59]:
print(npArr.shape)
#print(labelArr.shape)
print(len(indxArr))
print(testArr.shape)
print(len(indyArr))

(6385, 160, 19)
6385
(1738, 160, 19)
1738


In [60]:
# one-hot encode label
from keras.utils import to_categorical
enc_labelArr = to_categorical(labelArr)
enc_labelArr.shape

(6385, 22)

In [61]:
#saving and loading
np.save('npArr', npArr)
np.save('enc_labelArr', enc_labelArr)
np.save('indxArr', indxArr)
np.save('testArr', testArr)
np.save('indyArr', indyArr)



#npArr = np.load('npArr.npy')
#enc_labelArr = np.load('enc_labelArr.npy')
#indxArr = np.load('indxArr.npy')
#testArr = np.load('testArr.npy')
#indyArr = np.load('indyArr.npy')

In [142]:
print(np.isnan(np.sum(npArr)))
npArr

True


array([[[1.47345597e-01, 1.44226724e-01, 1.43087830e-01, ...,
         0.00000000e+00, 1.89424208e-09, 1.70055990e-09],
        [1.60306486e-01, 1.77104264e-01, 1.61159440e-01, ...,
         0.00000000e+00, 1.17674168e-09, 5.64717544e-10],
        [1.35275586e-01, 1.50035594e-01, 1.49557060e-01, ...,
         0.00000000e+00, 6.81040809e-10, 7.32118870e-10],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

       [[6.48820909e-02, 1.18077136e-01, 8.20440598e-02, ...,
         1.08087166e-09, 6.13634854e-10, 3.27523261e-09],
        [4.69869015e-02, 1.12935200e-01, 5.35557489e-02, ...,
         6.09639444e-10, 4.68691372e-10, 1.45073843e-09],
        [1.03961372e-01, 

In [138]:
#train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(npArr, enc_labelArr, test_size = 0.2, random_state=7)
print(X_train.shape)
print(y_train.shape)

(5108, 160, 19)
(5108, 22)


### 1D Conv Model

In [63]:
#specify hyperparameters
num_classes = 22

In [66]:
# create CNN structure
model = models.Sequential()
model.add(layers.Conv1D(32, kernel_size=10, activation='relu', input_shape=(npArr.shape[1],npArr.shape[2])))
model.add(layers.SpatialDropout1D(rate=0.2))
#model.add(layers.BatchNormalization())

model.add(layers.Conv1D(64, kernel_size=5, activation='relu'))
model.add(layers.SpatialDropout1D(rate=0.4))
#model.add(layers.BatchNormalization())

#model.add(layers.Conv1D(256, kernel_size=4, activation='relu'))
#model.add(layers.SpatialDropout1D(rate=0.4))
#model.add(layers.BatchNormalization())

model.add(layers.Conv1D(128, kernel_size=4, activation='relu'))
model.add(layers.SpatialDropout1D(rate=0.4))
#model.add(layers.BatchNormalization())

#model.add(layers.Conv1D(64, kernel_size=4, activation='relu'))
#model.add(layers.SpatialDropout1D(rate=0.4))
#model.add(layers.BatchNormalization())

model.add(layers.Flatten())
model.add(layers.Dense(5000, activation = 'relu'))
model.add(layers.Dropout(rate=0.5))
model.add(layers.Dense(num_classes, activation= 'softmax'))
callbacks = [EarlyStopping(monitor="val_loss", patience=10), ModelCheckpoint(filepath="best_cnn_model.h5", 
                                                                           monitor="val_loss",save_best_only=True)]

adam = optimizers.Adam(lr = 0.001, decay = 0.0001)

#compile model using accuracy to measure model performance
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [67]:
#specify hyperparameters
epochs = 200
batch_size = 32

history = model.fit(npArr, enc_labelArr, epochs=epochs, verbose=1, batch_size=batch_size, 
                      callbacks=callbacks, validation_split = 0.2)

Train on 5108 samples, validate on 1277 samples
Epoch 1/200
 416/5108 [=>............................] - ETA: 3:45 - loss: nan - acc: 0.0409

KeyboardInterrupt: 

### Maybe 2D Conv Model?
maybe 1D Conv Layer then a couple 2D Conv?

### Regular MLP

In [13]:
#load dataset
feature_set = pd.read_csv(dataFolder + 'new_feature_df.csv')
feature_set.replace(np.NaN, 0.0, inplace=True)
feature_set.replace(np.inf, 4.0, inplace=True)

# get trainDf again if top isn't run fully
trainDf = pd.read_csv(dataFolder+trainData)
trainDf = trainDf[trainDf.Label != 'lay']

#merge labels to get training set
feature_set = pd.merge(feature_set, trainDf, left_on='id', right_on = 'Datafile', how = 'inner')
fileorder = feature_set['id']

# one-hot encode label
from keras.utils import to_categorical
factor = pd.factorize(feature_set.Label)
feature_set.Label = factor[0]
labels = to_categorical(feature_set['Label'].values)

feature_set.drop(columns= ['id', 'Subject', 'Datafile'], inplace = True)


#normalize feature set
f = feature_set.values.astype('float32')
f = normalize(f, axis = 1)
f[:5]

array([[ 1.6515067e-02, -6.7485064e-02, -4.0974207e-02, ...,
         3.4078300e-07, -2.1590799e-06,  0.0000000e+00],
       [ 2.1176578e-02, -2.7541181e-02, -4.2534634e-02, ...,
        -6.9564326e-06,  4.9421424e-06,  9.0536887e-06],
       [ 5.3427730e-02, -1.5425559e-02, -2.5265059e-02, ...,
        -1.6132236e-06,  7.0478754e-06,  2.3184088e-05],
       [ 5.1594611e-02, -3.2933217e-02, -8.1078112e-02, ...,
        -6.1627206e-06, -1.0733088e-06,  7.5541160e-05],
       [ 2.4223570e-02, -3.6726356e-02, -3.2027334e-02, ...,
         1.7665525e-06,  7.1124373e-07,  3.2856657e-05]], dtype=float32)

In [58]:
# create MLP structure
mlp = models.Sequential()
mlp.add(layers.Dense(2640, activation='relu', input_dim=264))
mlp.add(layers.Dropout(rate=0.2))
mlp.add(layers.Dense(1280, activation = 'relu'))
mlp.add(layers.Dropout(rate=0.5))
mlp.add(layers.Dense(800, activation = 'relu'))
mlp.add(layers.Dropout(rate=0.5))
mlp.add(layers.Dense(400, activation = 'relu'))
mlp.add(layers.Dropout(rate=0.5))
mlp.add(layers.Dense(22, activation= 'softmax'))

callbacks = [EarlyStopping(monitor="val_loss", patience=10), ModelCheckpoint(filepath="best_mlp_model.h5", 
                                                                           monitor="val_loss",save_best_only=True)]

adam = optimizers.Adam(lr = 0.001)
sgd = optimizers.SGD(lr = 0.001, momentum = 0.1, decay = 0.001)
#compile model using accuracy to measure model performance
mlp.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [59]:
epochs = 200

batch_size = 32

hist = mlp.fit(f, labels, batch_size=batch_size, epochs=epochs, callbacks=callbacks, 
          validation_split=0.2)

Train on 5108 samples, validate on 1277 samples
Epoch 1/200
5108/5108 [==============================] - 14s 3ms/step - loss: 2.5054 - acc: 0.1754 - val_loss: 2.0944 - val_acc: 0.2553
Epoch 2/200
5108/5108 [==============================] - 11s 2ms/step - loss: 1.6522 - acc: 0.3604 - val_loss: 1.8482 - val_acc: 0.3547
Epoch 3/200
5108/5108 [==============================] - 11s 2ms/step - loss: 1.2091 - acc: 0.4955 - val_loss: 1.2940 - val_acc: 0.4894
Epoch 4/200
5108/5108 [==============================] - 12s 2ms/step - loss: 0.9715 - acc: 0.5781 - val_loss: 1.3999 - val_acc: 0.5129
Epoch 5/200
5108/5108 [==============================] - 12s 2ms/step - loss: 0.8481 - acc: 0.6341 - val_loss: 1.1263 - val_acc: 0.5873
Epoch 6/200
5108/5108 [==============================] - 11s 2ms/step - loss: 0.8224 - acc: 0.6443 - val_loss: 1.4963 - val_acc: 0.5411
Epoch 7/200
5108/5108 [==============================] - 12s 2ms/step - loss: 0.6800 - acc: 0.7034 - val_loss: 1.2380 - val_acc: 0.5442


### CNN plus feature MLP
inspired by https://www.pyimagesearch.com/2019/02/04/keras-multiple-inputs-and-mixed-data/

In [32]:
#load dataset
feature_set = pd.read_csv(dataFolder + 'new_feature_df.csv')
feature_set.replace(np.NaN, 0.0, inplace=True)
feature_set.replace(np.inf, 4.0, inplace=True)

# get trainDf again if top isn't run fully
trainDf = pd.read_csv(dataFolder+trainData)
trainDf = trainDf[trainDf.Label != 'lay']

#merge labels to get training set
feature_set = pd.merge(feature_set, trainDf, left_on='id', right_on = 'Datafile', how = 'inner')
fileorder = feature_set['id']

feature_set.drop(columns= ['id', 'Subject', 'Datafile', 'Label'], inplace = True)

#normalize feature set
f = feature_set.values
f = (f - f.min(axis=1, keepdims=True)) / (f.max(axis=1, keepdims=True) - f.min(axis=1, keepdims=True))

In [127]:
### Network Structure
# import the necessary packages
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers.core import SpatialDropout1D
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Concatenate
from keras.layers import concatenate

#input_ts = Input(shape=(npArr.shape[1], npArr.shape[2]))
#input_f = Input(shape=f_train.shape[1],)

In [6]:
## creating CNN component
# initialize the input shape and channel dimension, assuming
# TensorFlow/channels-last ordering
    
inputShape = (npArr.shape[1], npArr.shape[2],)
chanDim = -1
 
# define the model input
input_ts = Input(shape=inputShape)
 
# CONV => RELU => BN => POOL
c = Conv1D(32, 10, strides=10, activation = 'relu')(input_ts)
c = BatchNormalization(axis=chanDim, momentum = 0.0)(c)
c = MaxPooling1D(pool_size=2)(c)
c = SpatialDropout1D(rate=0.2)(c)
    
c = Conv1D(64, 10, activation = 'relu')(c)
c = BatchNormalization(axis=chanDim, momentum = 0.0)(c)
c = MaxPooling1D(pool_size=2)(c)
c = SpatialDropout1D(rate=0.2)(c)
    
c = Conv1D(128, 4, activation = 'relu')(c)
c = BatchNormalization(axis=chanDim, momentum = 0.0)(c)
c = MaxPooling1D(pool_size=2)(c)
c = SpatialDropout1D(rate=0.2)(c)
        
# flatten the volume, then FC => RELU => BN => DROPOUT
c = Flatten()(c)
c = Dense(2000, activation='relu')(c)
c = Dropout(0.4)(c)
 
# apply another FC layer, this one to match the number of nodes
# coming out of the MLP
c_out = Dense(500, activation='relu')(c)

### create simple MLP
#model input
input_m = Input(shape=(264,))

#two dense layers with dropout in between
m = Dense(1000, activation="relu")(input_m)
m = BatchNormalization(axis=chanDim, momentum = 0.0)(m)
m = Dropout(rate=0.2)(m)
m = Dense(1000, activation="relu")(input_m)
m = Dropout(rate=0.4)(m)
m_out = Dense(500, activation="relu")(m)

# create the input to our final set of layers as the *output* of both
# the MLP and CNN
comb = concatenate([c_out, m_out])

# our final FC layer head will have two dense layers, the final one
# being our regression head
comb = Dense(1000, activation="relu")(comb)
comb = Dropout(rate = 0.5)(comb)
out = Dense(22, activation="softmax")(comb)

model = Model(inputs=[input_ts, input_m], outputs=out)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
#specify optimizers
adam = Adam(lr = 0.001, decay = 0.001 / 200)

#compile model using accuracy to measure model performance
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
#specify hyperparameters
epochs = 200
batch_size = 64

#early stopping
callbacks = [EarlyStopping(monitor="val_loss", patience=30), ModelCheckpoint(filepath="best_comb_model.h5", 
                                                                           monitor="val_loss",save_best_only=True)]

model.fit([npArr, f], enc_labelArr, batch_size=batch_size, epochs=epochs, callbacks=callbacks, 
          validation_split=0.2, shuffle = True)

Instructions for updating:
Use tf.cast instead.
Train on 5108 samples, validate on 1277 samples
Epoch 1/200
5108/5108 [==============================] - 65s 13ms/step - loss: 2.5863 - acc: 0.2101 - val_loss: 1.3896 - val_acc: 0.4839
Epoch 2/200
5108/5108 [==============================] - 63s 12ms/step - loss: 1.3617 - acc: 0.5305 - val_loss: 0.9725 - val_acc: 0.6711
Epoch 3/200
5108/5108 [==============================] - 63s 12ms/step - loss: 0.8792 - acc: 0.6962 - val_loss: 0.7243 - val_acc: 0.7439
Epoch 4/200
5108/5108 [==============================] - 63s 12ms/step - loss: 0.6472 - acc: 0.7892 - val_loss: 0.6493 - val_acc: 0.7886
Epoch 5/200
5108/5108 [==============================] - 63s 12ms/step - loss: 0.5158 - acc: 0.8289 - val_loss: 0.8706 - val_acc: 0.7619
Epoch 6/200
5108/5108 [==============================] - 63s 12ms/step - loss: 0.4650 - acc: 0.8496 - val_loss: 0.8612 - val_acc: 0.7776
Epoch 7/200
5108/5108 [==============================] - 63s 12ms/step - loss: 0.4

KeyboardInterrupt: 

### Turning sensor time series into graphs and running CNN on the images

In [11]:
# load feature df to look at mins/maxs for images
feature_df = pd.read_csv(mainFolder + 'new_feature_df.csv')

feature_df.head()

,id,n_rows,emg1_min,emg2_min,emg3_min,emg4_min,airborne_min,acc_u_x_min,acc_u_y_min,acc_u_z_min,...,cor_gyro_u_y_gyro_u_z,cor_gyro_u_y_gyro_l_x,cor_gyro_u_y_gyro_l_y,cor_gyro_u_y_gyro_l_z,cor_gyro_u_z_gyro_l_x,cor_gyro_u_z_gyro_l_y,cor_gyro_u_z_gyro_l_z,cor_gyro_l_x_gyro_l_y,cor_gyro_l_x_gyro_l_z,cor_gyro_l_y_gyro_l_z
0,Subject05/Subject05_Aufnahme328.csv,1879,-7678.106440,-4661.836615,-3048.633316,-7512.425226,-6532.705695,-6284.303353,-6827.939329,-16398.609367,...,-0.105410,0.253597,-0.779391,0.084289,-0.158099,0.147542,0.310263,-0.257414,0.038773,-0.245649
1,Subject05/Subject05_Aufnahme171.csv,2339,-3041.984609,-4698.044891,-9783.315092,-9683.179991,-26136.890552,-6783.916204,-3552.906370,-7639.485250,...,0.014532,0.021924,-0.099626,-0.106292,-0.809414,0.554187,0.849509,-0.389958,-0.768353,0.545871
2,Subject05/Subject05_Aufnahme075.csv,4609,-1330.702322,-2179.517032,-4327.324365,-5921.050770,-18257.753309,-2382.916468,-2659.078325,-2702.537644,...,0.040820,-0.340181,-0.094396,-0.290452,-0.254656,0.746048,0.724259,-0.186918,-0.139166,0.607992
3,Subject05/Subject05_Aufnahme422.csv,2049,-1307.891654,-3219.891654,-2915.438263,-1877.487555,-3637.914592,-3448.846266,-1746.390434,-6552.304539,...,0.264991,0.152688,0.157523,0.106624,-0.169769,0.270967,-0.013500,0.115817,-0.244743,-0.042625
4,Subject05/Subject05_Aufnahme407.csv,2949,-4471.100373,-3899.037640,-3192.797898,-9610.212614,-26111.219736,-7406.349949,-7765.860292,-7300.351306,...,-0.468957,-0.166220,-0.675509,-0.223699,0.178065,0.379014,0.579796,-0.310766,0.215062,0.086587


In [13]:
feature_df.columns[:50]

Index(['id', 'n_rows', 'emg1_min', 'emg2_min', 'emg3_min', 'emg4_min',
       'airborne_min', 'acc_u_x_min', 'acc_u_y_min', 'acc_u_z_min',
       'gonio_x_min', 'acc_l_x_min', 'acc_l_y_min', 'acc_l_z_min',
       'gonio_y_min', 'gyro_u_x_min', 'gyro_u_y_min', 'gyro_u_z_min',
       'gyro_l_x_min', 'gyro_l_y_min', 'gyro_l_z_min', 'emg1_max', 'emg2_max',
       'emg3_max', 'emg4_max', 'airborne_max', 'acc_u_x_max', 'acc_u_y_max',
       'acc_u_z_max', 'gonio_x_max', 'acc_l_x_max', 'acc_l_y_max',
       'acc_l_z_max', 'gonio_y_max', 'gyro_u_x_max', 'gyro_u_y_max',
       'gyro_u_z_max', 'gyro_l_x_max', 'gyro_l_y_max', 'gyro_l_z_max',
       'emg1_std', 'emg2_std', 'emg3_std', 'emg4_std', 'airborne_std',
       'acc_u_x_std', 'acc_u_y_std', 'acc_u_z_std', 'gonio_x_std',
       'acc_l_x_std'],
      dtype='object')

In [78]:
max_list = list(feature_df.columns[21:40])
max_values = list(feature_df[max_list].max())
max_values

[33941.816560894535,
 32833.65760555235,
 32854.93024585477,
 32973.58286629304,
 52745.637340496985,
 22873.599682287528,
 21567.1392659628,
 23893.306845380568,
 10979.977144998127,
 37023.81536293164,
 38556.35982339956,
 37217.67458603312,
 10942.669613773847,
 37525.20099255583,
 39835.59878154917,
 40473.19206492335,
 37637.143934201515,
 40257.900990099006,
 39148.57755539671]

In [79]:
# get a list of all columns that contain minimums for all observations
min_list = list(feature_df.columns[2:21])
min_values = list(feature_df[min_list].min())
min_values

[-32915.482561463694,
 -33119.311906092786,
 -32868.74304322084,
 -62068.8383862497,
 -32972.93692178301,
 -25044.719731625497,
 -28415.224992490235,
 -27177.726172912364,
 -9310.37669059943,
 -29776.421763304214,
 -31110.498317528294,
 -30268.897595034912,
 -11102.855388813095,
 -32316.157631359467,
 -32177.498635122836,
 -33929.99018645731,
 -34299.19911012236,
 -33714.875364735315,
 -32748.26306465899]

In [119]:
# Adaptation of Prateeks code for generating dataset

# Number of files to load
numFiles = 6385

# Maximum file size
maxlen = 8000

#hyperparams
window_length = 100
# n_windows = 8000 / window_length
n_windows = 80
# number of intervals for binning
n_bins = 500

# NP Arrays to store the data and the label
labelArr = np.empty((numFiles, 1))
imgArr = np.empty((numFiles, n_bins, n_windows, 19))
indexArr = []

# read the training data into a dataframe
trainDf = pd.read_csv(dataFolder+trainData)
trainDf = trainDf[trainDf.Label != 'lay']
# factorize the labels
factor = pd.factorize(trainDf.Label)
trainDf.Label = factor[0]


cnt = 0
for eachFile in result:
  # Only read the dataset with labels available
    if sum(str(eachFile)[-35:] == trainDf.Datafile)>0:
        indexArr.append(str(eachFile)[-35:])
        file = np.loadtxt(str(eachFile), delimiter=',')
        #min max scale the input
        
        
        #zero pad to conserve last values
        padFile = np.zeros((8000, 19))
        file = np.concatenate((file, padFile))
        file = file[:8000,:]
        
        #creates windows by reshaping, creates 80 windows of 100 observations each
        file = file.reshape(n_windows, window_length, 19)
        
        #initializing empty binned file
        bfile = np.empty((n_bins, n_windows, 19))
        
        #loop over sensors and windows to put each observation into a bin for each window
        for sens in range(file.shape[2]):
            #create interval index for bins from min and max values
            bins = pd.cut(np.array([min_values[sens], max_values[sens]]), bins = bfile.shape[0]).categories
            for window in range(file.shape[0]):
                #get the counts for each bin in descending order and insert into new file
                bfile[:,window,sens] = list(pd.cut(file[window,:,sens], bins=bins).value_counts())[::-1]
        
        #normalize bfile by dividing all counts by 100 (n_obs in each window)
        bfile = bfile / window_length
        data = np.expand_dims(bfile, axis=0)
        imgArr[cnt] = data
        labelArr[cnt] = trainDf[trainDf.Datafile == str(eachFile)[-35:]].Label
        cnt += 1

In [120]:
imgArr = npArr
indexArr = indxArr

In [121]:
imgArr.shape

(6385, 500, 80, 19)

In [125]:
np.save('imgArr_500b_80w', imgArr)
np.save('labelArr', labelArrd)

In [ ]:
imgArr = np.load('imgArr_500b_80w.npy')


In [130]:
# one-hot encode label
from keras.utils import to_categorical
enc_labelArr = to_categorical(labelArr)
enc_labelArr.shape

(6385, 22)

In [131]:
print(np.isnan(np.sum(imgArr)))

False


#### Image Creation comments
Next time initialize bfile as zeros and don't do any padding, will just fill the entries that the file has. Have to look further into looping and such but would save time when creating dataset <br>
Also should either do smaller bins or larger windows so dimensions are somewhat similar

In [128]:
# import the necessary packages for the NN structure
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers.core import SpatialDropout2D
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from keras.models import Model
from keras.optimizers import Adam

In [129]:
inputShape = (imgArr.shape[1], imgArr.shape[2], imgArr.shape[3])
chanDim = -1
 
# define the model input
imginput = Input(shape=inputShape)

In [151]:
# CONV => RELU => BN => POOL
cnn = Conv2D(32, (10, 4), activation = 'relu', use_bias=True)(imginput)
#cnn = BatchNormalization(axis=chanDim)(cnn)
#cnn = SpatialDropout2D(rate=0.2)(cnn)
    
cnn = Conv2D(64, 4, activation = 'relu', use_bias = True)(cnn)
#cnn = BatchNormalization(axis=chanDim)(cnn)
#cnn = SpatialDropout1D(rate=0.2)(cnn)
    
cnn = Conv2D(32, 4, activation = 'relu', use_bias=True)(cnn)
#cnn = BatchNormalization(axis=chanDim)(cnn)
#cnn = SpatialDropout1D(rate=0.2)(cnn)
        
# flatten it for predictions
cnn = Flatten()(cnn)
cnn = Dense(2000, activation='relu')(cnn)
#cnn = Dropout(0.4)(cnn)
out = Dense(22, activation='softmax')(cnn)

In [152]:
adam = optimizers.Adam(lr = 0.001, decay = 0.0000001)

#compile model using accuracy to measure model performance
model = Model(inputs=imginput, outputs=out)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#specify hyperparameters
epochs = 200
batch_size = 100

#early stopping
callbacks = [EarlyStopping(monitor="val_loss", patience=10), ModelCheckpoint(filepath="best_img_model.h5", 
                                                                           monitor="val_loss",save_best_only=True)]

model.fit(imgArr, enc_labelArr, batch_size=batch_size, epochs=epochs, callbacks=callbacks, 
          validation_split=0.2)

Instructions for updating:
Use tf.cast instead.
Train on 5108 samples, validate on 1277 samples
Epoch 1/200
